## predict fight outcome on current game state  
- current ults available per team
- teammates alive
- roles alive
- previous fights of map/round
- include map, some roles maybe more influential given map

UltsAvail = # ult rows where time in [kill1, kill2]
will have to go map by map, scrim by scrim

using primarily Kill, also UltimateCharged

### Notes:
- we can look at only attacker team win%, because defender is 1-a % win
- it's probably best to omit the identity of a player, so their performance relative to entered scrims does not skew

### collection steps:  
1. get kills data
2. number fights per map
3. add ults
4. re-index on time
5. calculate teammates alive
6. calculate n roles alive

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import db_connection as db
import cleaning as cdb

load_dotenv(".env")
url = os.getenv("prod_url")

conn = db.createConnection(db.parseURL(url))

killQuery = """
SELECT "match_time",
    "event_type",
    "attacker_team",
    "attacker_hero",
    "attacker_name",
    "victim_team",
    "victim_hero",
    "victim_name",
    "MapDataId"
FROM "Kill"
"""

killDB = db.sqlSearch(conn, killQuery)
killDB.head()

,match_time,event_type,attacker_team,attacker_hero,attacker_name,victim_team,victim_hero,victim_name,MapDataId
0,19.73,kill,ASU,Junker Queen,Aurora,SCC,Kiriko,Reyzr,20.0
1,22.16,kill,SCC,Junker Queen,Tred,ASU,Sojourn,Jman,20.0
2,29.78,kill,ASU,Kiriko,zPanthr,SCC,Sojourn,Noxious,20.0
3,37.71,kill,ASU,Genji,Anhoo,SCC,Genji,Emmeryn,20.0
4,41.28,kill,ASU,Kiriko,zPanthr,SCC,Lúcio,redex,20.0


In [2]:
def calculate_fight_ids(group):
    group['time_prev'] = group['match_time'].diff()
    group['time_next'] = group['time_prev'].shift(-1)
    group['is_stagger'] = (group['time_prev'] > 15) & (group['time_next'] > 15)
    group['fight_id'] = group.loc[~group['is_stagger'], 'time_prev'].gt(15).cumsum() + 1
    group['fight_id'] = group['fight_id'].fillna(-1).astype(int)
    return group

killDB = killDB.groupby('MapDataId').apply(calculate_fight_ids)

killDB.head(5)


C:\Users\Xavier\AppData\Local\Temp\ipykernel_6636\3575742219.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  killDB = killDB.groupby('MapDataId').apply(calculate_fight_ids)


match_time event_type attacker_team attacker_hero attacker_name  \
MapDataId                                                                      
20.0      0       19.73       kill           ASU  Junker Queen        Aurora   
          1       22.16       kill           SCC  Junker Queen          Tred   
          2       29.78       kill           ASU        Kiriko       zPanthr   
          3       37.71       kill           ASU         Genji         Anhoo   
          4       41.28       kill           ASU        Kiriko       zPanthr   

            victim_team victim_hero victim_name  MapDataId  time_prev  \
MapDataId                                                               
20.0      0         SCC      Kiriko       Reyzr       20.0        NaN   
          1         ASU     Sojourn        Jman       20.0       2.43   
          2         SCC     Sojourn     Noxious       20.0       7.62   
          3         SCC       Genji     Emmeryn       20.0       7.93   
          4         SCC       Lúcio       redex       20.0       3.57   

             time_next  is_stagger  fight_id  
MapDataId                                     
20.0      0       2.43       False         1  
          1       7.62       False         1  
          2       7.93       False         1  
          3       3.57       False         1  
          4       1.33       False         1

get ult charges

In [3]:
ultQuery = """
SELECT "match_time",
    "event_type",
    "player_team",
    "player_hero",
    "player_name",
    "MapDataId"
FROM "UltimateCharged"
"""

ultDB = db.sqlSearch(conn, ultQuery)
ultDB.head(10)

,match_time,event_type,player_team,player_hero,player_name,MapDataId
0,91.24,ultimate_charged,ASU,Kiriko,zPanthr,20.0
1,97.55,ultimate_charged,SCC,Junker Queen,Tred,20.0
2,113.00,ultimate_charged,SCC,Lúcio,redex,20.0
3,127.47,ultimate_charged,SCC,Kiriko,Reyzr,20.0
4,130.83,ultimate_charged,ASU,Genji,Anhoo,20.0
5,140.38,ultimate_charged,SCC,Genji,Emmeryn,20.0
6,141.54,ultimate_charged,ASU,Junker Queen,Aurora,20.0
7,159.14,ultimate_charged,ASU,Sojourn,Jman,20.0
8,186.59,ultimate_charged,SCC,Sojourn,Noxious,20.0
9,211.71,ultimate_charged,ASU,Lúcio,Anghell1c,20.0
